<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/wion_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 요청과 응답
####요청
url = 'http://companyinfo.stock.naver.com/v1/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=0&finGubun=MAIN&frqTyp=0&cn=&encparam=SjBwNnVnakw3a1lmZ1BUVFdKcFdPUT09'
* cmp_cd (종목코드): '005930'=삼성전자
* frq (기간): '0'=연간, '1'=분기
* rpt (종류): '0'=손익계산서, '1'=재무상태표, '2'=현금흐름표
* finGubun (구분): 'MAIN'=주재무제표, 'IFRSS'=IFRS분리, 'IFRSL'=IFRS연결, 'GAAPS'=GAAP개별, 'GAAPL'=GAAP연결
* frqTyp (기간): '0'=연간, '1'=분기 (frq와 동일)
* cn (미사용)
* encparam: 암호화된 정보 (페이지에 포함)

In [ ]:
# 네이버 파이낸스 제무제표 상세

import re
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

def naver_finstate_detail(cmp_cd, rpt='0', frq='1', finGubun='IFRS'):
    '''
    네이버 파이낸스로 부터 상세재무제표를 읽어온다
    * cmp_cd:  # 종목코드: '005930'
    * rpt='0' # 종류: '0'=손익계산서(기본값), '1'=재무상태표, '2'=현금흐름표
    * frq='1' # 기간: '0'=연간, '1'=분기(기본값)
    * finGubun='IFRSL' # 구분: 'MAIN'=주재무제표, 'IFRSS'=KIFRS별도, 'IFRSL'=IFRS연결(기본값), 'GAAPS'=GAAP개별, 'GAAPL'=GAAP연결
    '''

    # encparam 가져오기
    url = 'http://companyinfo.stock.naver.com/v1/company/c1030001.aspx?cmp_cd=005930'
    html_text = requests.get(url).text
    encparam = re.findall ("encparam: '(.*?)'", html_text)[0]

    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/cF3002.aspx?' \
                    'cmp_cd={cmp_cd}&frq={frq}&rpt={rpt}&finGubun={finGubun}&frqTyp={frq}&cn=&encparam={encparam}'  

    url = url_tmpl.format(cmp_cd=cmp_cd, frq=frq, rpt=rpt, finGubun=finGubun, frqTyp=frq, encparam=encparam)

    # 페이지 가져오기
    headers={'Referer': url}
    jo = json.loads(requests.get(url, headers=headers).text)
    
    # DataFrame 생성
    df = json_normalize(jo, 'DATA')

    # DATA1~DATA6 컬럼 이름 바꾸기 
    jo_yymm = jo['YYMM'][:6]
    date_str_list = []
    for yymm in jo_yymm:
        m = re.search('(\d{4}/\d{0,2}).*', yymm)
        date_str_list.append(m.group(1) if m else '') 
    data_n_list = ['DATA' + str(i) for i in range(1,7)]
    yymm_cols = zip(data_n_list, date_str_list)
    cols_map = dict(yymm_cols)
    df.rename(columns=cols_map, inplace=True)
    df['ACC_NM'] = df['ACC_NM'].str.strip().replace('[\.*\[\]]', '', regex=True)
    df.set_index(['ACCODE', 'ACC_NM'], inplace=True)
    df = df.iloc[:, 5:10] # 날짜 컬럼만 추출
    df = df.T # Transpose (컬럼, 인덱스 바꾸기)
    df.index = pd.to_datetime(df.index)
    df.index.name = '날짜'
    return df

In [ ]:
IS = naver_finstate_detail('005930', rpt='0', frq='0')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [ ]:
BS = naver_finstate_detail('005930', rpt='1', frq='0')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [ ]:
CF = naver_finstate_detail('005930', rpt='2', frq='0')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [ ]:
cd /content/drive/My Drive/wion_crawling

/content/drive/My Drive/wion_crawling


In [ ]:
IS.to_csv('IS.csv', sep = ',', na_rep = 'NaN')

In [ ]:
BS.to_csv('BS.csv', sep = ',', na_rep = 'NaN')

In [ ]:
CF.to_csv('CF.csv', sep = ',', na_rep = 'NaN')